In [1]:
import numpy as np
import pandas as pd
import os
import gc

data_path = "data"

# 1. INTRODUCTION

challange: huge amount of data
    

<b> check data </b><br>
$ ls -lh ./data
187M  2019 sample_submission.csv <br>
824M Dec 12  2019 test.csv <br>
2.5G Dec 12  2019 test_supplement.csv <br>
7.1G Dec 12  2019 train.csv <br>
3.9M Dec 12  2019 train_sample.csv <br>

$ cat train.csv | wc -l<br>
184903891

$ cat test.csv | wc -l<br>
18790470



use gc (garbage collector). I woulc be better to  delete constantly unncessary things.<br>
use dask instead of pandas <br>
read csv chunk by chunk or use feather <br>
use appropriate dyteps while reading file columns to reduce memory usage <br>

